In [1]:
import warnings
warnings.filterwarnings('ignore')

from nba_api.stats.endpoints import playercareerstats, commonplayerinfo
from nba_api.stats.static import players
from tqdm.notebook import tqdm

import pandas as pd

with open('config.txt', 'r') as f:
    names = f.read().split('\n')

def process_roster(path):
    roster = pd.read_csv(path)
    roster['Game'] = roster.Game.str.split('@').apply(lambda x: set([y.strip() for y in x]))
    roster['Name'] = roster.Name
    roster['TeamAbbrev'] = roster.TeamAbbrev.apply(lambda x: set([x]))
    roster['OppAbbrev'] = roster.Game - roster.TeamAbbrev
    roster['Team'] = roster.TeamAbbrev.apply(lambda x: list(x)[0])
    roster['Opp'] = roster.OppAbbrev.apply(lambda x: list(x)[0])
    roster.Time = pd.to_datetime(roster.Time)
    roster.drop(['TeamAbbrev','OppAbbrev', 'Game'], axis = 1, inplace=True)
    return roster[['Name','Position', 'ID', 'Time', 'Salary', 'Team', 'Opp']]

In [226]:
player_info.get_json()

'{"resource": "commonplayerinfo", "parameters": [{"PlayerID": 203999}, {"LeagueID": null}], "resultSets": [{"name": "CommonPlayerInfo", "headers": ["PERSON_ID", "FIRST_NAME", "LAST_NAME", "DISPLAY_FIRST_LAST", "DISPLAY_LAST_COMMA_FIRST", "DISPLAY_FI_LAST", "PLAYER_SLUG", "BIRTHDATE", "SCHOOL", "COUNTRY", "LAST_AFFILIATION", "HEIGHT", "WEIGHT", "SEASON_EXP", "JERSEY", "POSITION", "ROSTERSTATUS", "GAMES_PLAYED_CURRENT_SEASON_FLAG", "TEAM_ID", "TEAM_NAME", "TEAM_ABBREVIATION", "TEAM_CODE", "TEAM_CITY", "PLAYERCODE", "FROM_YEAR", "TO_YEAR", "DLEAGUE_FLAG", "NBA_FLAG", "GAMES_PLAYED_FLAG", "DRAFT_YEAR", "DRAFT_ROUND", "DRAFT_NUMBER", "GREATEST_75_FLAG"], "rowSet": [[203999, "Nikola", "Jokic", "Nikola Jokic", "Jokic, Nikola", "N. Jokic", "nikola-jokic", "1995-02-19T00:00:00", "Mega Basket", "Serbia", "Mega Basket/Serbia", "6-11", "284", 7, "15", "Center", "Active", "Y", 1610612743, "Nuggets", "DEN", "nuggets", "Denver", "nikola_jokic", 2015, 2022, "N", "Y", "Y", "2014", "2", "41", "N"]]}, {"

In [215]:
df = process_roster(f'data/NBA $3K AND-ONE [20 ENTRY MAX]/2022-10-10.csv').set_index('Name')
nba_id = pd.DataFrame(players.get_active_players()).set_index('full_name')['id'].astype(int).rename('NBA_ID')
df = df.join(nba_id)
df.head()

,Position,ID,Time,Salary,Team,Opp,NBA_ID
Name,,,,,,,
Nikola Jokic,C,830650,2022-10-11 01:00:00+00:00,10100.0,DEN,PHX,203999.0
Joel Embiid,C,794508,2022-10-10 23:00:00+00:00,9900.0,PHI,CLE,203954.0
James Harden,PG/SG,395388,2022-10-10 23:00:00+00:00,9600.0,PHI,CLE,201935.0
LaMelo Ball,PG,1257420,2022-10-10 23:00:00+00:00,9200.0,CHA,WAS,1630163.0
Devin Booker,PG/SG,845564,2022-10-11 01:00:00+00:00,9100.0,PHX,DEN,1626164.0


In [216]:
# Appendix
#Getting NBA Ids

# nba_id = {}
#
# for name in tqdm(pd.read_csv('data/player_names.csv').Player):
#     try:
#         id = players.find_players_by_full_name(name)[0]['id']
#         nba_id[name] = id
#     except Exception as e:
#         print(f'failed for {name}')
#         print(e)
#
#     import json
# with open('data/nba_id.json', 'w') as f:
#     json.dump(nba_id, f)